Ładujemy pakiety

In [1]:
import matplotlib.pyplot as plt
from matplotlib.backend_bases import RendererBase
from scipy import signal
from scipy.io import wavfile
#import soundfile as sf
import os
import numpy as np
from PIL import Image
from scipy.fftpack import fft
from google.colab import drive
import librosa

%matplotlib inline

Połączenie z Drive

In [2]:
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
audio_path = '/content/drive/My Drive/01_Dane/01_Dzwieki'
pict_Path = '/content/drive/My Drive/01_Dane/02_Spektrogramy'
samples = []

Wyciągamy nazwy podfolderów - gatunków ptaków

In [4]:
subFolderList = []
for x in os.listdir(audio_path):
    if os.path.isdir(audio_path + '/' + x):
        subFolderList.append(x)

Tworzymy odpowiednie połączenia do ścieżek

In [6]:
if not os.path.exists(pict_Path):
    os.makedirs(pict_Path)

subFolderList = []
for x in os.listdir(audio_path):
    if os.path.isdir(audio_path + '/' + x):
        subFolderList.append(x)
        if not os.path.exists(pict_Path + '/' + x):
            os.makedirs(pict_Path +'/'+ x)

Tworzymy ścieżki każdego pliku dźwiękowego i liczymy, ile mamy plików per gatunek

In [ ]:
sample_audio = []
total = 0
for x in subFolderList:
        
    all_files = [y for y in os.listdir(audio_path + '/' + x) if '.mp3' in y]
    total += len(all_files)

    sample_audio.append(audio_path + '/' + x + '/'+ all_files[0])
    
    print('count: %d : %s' % (len(all_files), x ))
    
print(total)

Tworzymy funkcję generującą spektrogram

In [ ]:
def log_specgram(audio, sample_rate, window_size=20,
                 step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, _, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, np.log(spec.T.astype(np.float32) + eps)

6 przykładowych spektrogramów

In [ ]:
fig = plt.figure(figsize=(10,10))

for i, filepath in enumerate(sample_audio[:9]):
    
    plt.subplot(3,3,i+1)
    
    label = filepath.split('/')[-2]
    plt.title(label)
    
    test_sound, samplerate = librosa.load(filepath, res_type='kaiser_fast')
    print(samplerate)
    _, spectrogram = log_specgram(test_sound, samplerate)
    
    plt.imshow(spectrogram.T, aspect='auto', origin='lower')
    plt.axis('off')

    

6 przykładowych waveformów

In [ ]:
fig = plt.figure(figsize=(8,20))
for i, filepath in enumerate(sample_audio[:6]):
    plt.subplot(9,1,i+1)
    test_sound, samplerate = librosa.load(filepath, res_type='kaiser_fast')
    plt.title(filepath.split('/')[-2])
    plt.axis('off')
    plt.plot(test_sound)

Konwersja spektrogramu do .png

In [ ]:
def wav2img(wav_path, targetdir='', figsize=(4,4)):

    fig = plt.figure(figsize=figsize)    
    
    test_sound, samplerate = librosa.load(filepath, res_type='kaiser_fast')
    _, spectrogram = log_specgram(test_sound, samplerate)
    
    output_file = wav_path.split('/')[-1].split('.mp3')[0]
    output_file = targetdir +'/'+ output_file

    plt.imsave('%s.png' % output_file, spectrogram)
    plt.close()

Konwersja waveform do .png

In [ ]:
def wav2img_waveform(wav_path, targetdir='', figsize=(4,4)):

    fig = plt.figure(figsize=figsize)

    test_sound, samplerate = librosa.load(filepath, res_type='kaiser_fast')
    
    plt.plot(test_sound)
    plt.axis('off')
    output_file = wav_path.split('/')[-1].split('.mp3')[0]
    output_file = targetdir +'/'+ output_file
    
    plt.savefig('%s.png' % output_file)
    plt.close()

Pętla zapisująca pliki .png

In [ ]:
for i, x in enumerate(subFolderList):
    print(i, ':', x)
    
    all_files = [y for y in os.listdir(audio_path + '/' + x) if '.mp3' in y]
    for file in all_files[:10]:
        wav2img(audio_path + '/' + x + '/' + file, pict_Path + '/' + x)

In [ ]:
#https://www.kaggle.com/kubalakrystian/audio-data-conversion-to-images-eda/edit